In [1]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

#lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['LINKUSDT', 'LTCUSDT', 'BCHUSDT', 'WLDUSDT', 'IDUSDT', 'HBARUSDT', 'API3USDT', 'BNBUSDT', 'C98USDT', '1000SHIBUSDT', 'STXUSDT', 'LPTUSDT', 'XLMUSDT', 'CRVUSDT', 'MKRUSDT', 'ALGOUSDT', 'HIGHUSDT', 'COMPUSDT', 'DOGEUSDT', 'XRPUSDT', 'OPUSDT', 'ARBUSDT', 'AUDIOUSDT', '1000PEPEUSDT', 'MATICUSDT', 'APEUSDT', 'SOLUSDT', 'TOMOUSDT', 'EOSUSDT', 'ADAUSDT', 'SUIUSDT', 'YGGUSDT', 'DODOXUSDT']
lista=['ASTRUSDT']

if len(lista)>1:
    imprimo=False
else:
    imprimo= True
for symbol in lista:    
    try:
        data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data = md.estrategia_triangulos(symbol, tp_flag = False, print_lines_flag = imprimo)
        #data,porcentajeentrada = md.estrategia_adrian(symbol,tp_flag = False)
        #######################################################################
        #resultado = md.backtesting(data, plot_flag = imprimo)
        resultado = md.backtestingsanta(data, plot_flag = True,debug = True)
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Candleid-1: 324 - linea superior 0.043890265486725664
Candleid-1: 324 - linea inferior 0.04429999999999999
Precio Close-1: 0.0443
Candleid-1: 653 - linea superior 0.05425164835164838
Candleid-1: 653 - linea inferior 0.05500701754385975
Precio Close-1: 0.0529
Candleid-1: 654 - linea superior 0.05414505494505496
Candleid-1: 654 - linea inferior 0.055138596491228174
Precio Close-1: 0.0522
Candleid-1: 655 - linea superior 0.05403846153846156
Candleid-1: 655 - linea inferior 0.0552701754385966
Precio Close-1: 0.0521


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


ASTRUSDT - Return [%]: -0.14% - # Trades: 2 - Profit Factor: 0.0 - Win Rate [%]: 0.0


In [2]:
md.dibuja_patrones_triangulos (data,323) 

0.9030381159725251 -0.8435944233915904


linea superior. X: [303. 309. 313. 320. 335.] - y: [0.90682782 0.86373486 0.83500622 0.7847311  0.67699869]
linea inferior. X: [310. 316. 321. 336.] - y: [0.72849011 0.75204176 0.77166813 0.83054725]
